In [1]:
import torch
# from model import RouteNet
from CustomLoss import CustomLoss
from SimpleNetworkDataset import NetworkDataset,get_dataloader

In [2]:
fpath = "/home/steve/Documents/GitHub/CSE222a/CSE222RouteNet/data/pt_dir"

In [3]:
data_set = NetworkDataset(fpath)

In [4]:
dloader = get_dataloader(data_set,1)

In [5]:
def train(model,x,label,n_packets):
    optimizer = torch.optim.Adam(model.parameters(), .001) # optimizer method for gradient descent
#     criterion = torch.nn.MSELoss() 
    criterion = CustomLoss()
    model.train() #put model in training mode
    for epoch in range(100):
        tr_loss = []
        optimizer.zero_grad()  
        outputs = model(x)
        #the input of the parameters should be re-defined here
        loss = criterion(outputs, label,n_packets)
        loss.backward()                        
        optimizer.step()                  
        tr_loss.append(loss.item())
        print(tr_loss)

In [6]:
for _,batch in enumerate(dloader):
    x,y = batch
    break
    

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

'''using the implementation from RouteNet'''
class RouteNet(nn.Module):

    def __init__(self):
        super(RouteNet,self).__init__()

        ### Architecture ###
        # for gru need to pay attention to if input is of size:
        # (batch, seq_len, feature size) or seq_len, batch, feature size
        # if sequence length is variable
        # may need to pad the sequence
        
        self.link_state_dim = 32
        self.path_state_dim = 32
        self.readout_dim = 8
        self.output_units = 2
        self.T = 8

        inSize = 10 # place holder
        hSize  = 32
        readSize = 10
        nLayer = 1

        self.l_U = nn.GRU(input_size = self.link_state_dim,
                          hidden_size = hSize,
                          num_layers = nLayer,
                          batch_first=True)
        
        self.p_U = nn.GRU(input_size = self.path_state_dim,
                          hidden_size = hSize,
                          num_layers = nLayer,
                          batch_first=True)
        
        self.readOut = nn.ModuleDict({  'r1': nn.Linear(hSize,self.readout_dim),
                                        'r2': nn.Linear(self.readout_dim,self.readout_dim),
                                        'r3': nn.Linear(self.readout_dim,self.output_units)
                                        })
        

    def forward(self,x):
        
        links = torch.unsqueeze(torch.tensor(x['links']),1)                         
        paths = torch.unsqueeze(torch.tensor(x['paths']),1)                 
        seqs = torch.unsqueeze(torch.tensor(x['sequences']),1)
        link_cap = torch.unsqueeze(torch.tensor(x['link_capacity']).float(),axis=1)
        link_cap = link_cap/torch.max(link_cap)
        bandwidth = torch.unsqueeze(torch.tensor(x['bandwith']).float(), axis=1)
        bandwidth = bandwidth/torch.max(bandwidth)

        # state matrix shape for the link
        link_h_state_shape = (x['n_links'][0], self.link_state_dim-1)

        # create hidden state matrix shape for the path  
        path_h_state_shape = (x['n_paths'],self.path_state_dim-1)
        path_h_state = torch.cat((bandwidth,torch.zeros(path_h_state_shape)), axis=1)
        
        # prepare input for path update RNN
        max_seq_len = torch.max(seqs)
        path_rnn_input_shape = (x['n_paths'],max_seq_len+1,self.link_state_dim)
        
        #stack the paths and sequences
        ids = torch.stack((paths,seqs),axis=1)
        ids = torch.squeeze(ids,2)           
        p_ind = ids[:,0]
        s_ind = ids[:,1]

        # flatten the double loop into a bulk matrix using the gather functionality
        # this is an aggregation of the state vector of each link on each path flattened into a (sum_paths(sum_links_on_paths) x h_state) size matrix 
        # using torch.gather
        indices = torch.zeros(len(links),32)
        for i in range(len(links)):
            link_id = links[i]
            indices[i,:] = link_id
            
        # variable dictionary for forward pass
        vd = {}
        
        for t in range(self.T):
            
            
            ############# set up the matrices and variables for each pass through #################
            
            ########## PATH VARIABLES ###########
            
            # input to the path rnn layer P_u
            path_rnn_input_key = 'path_rnn_input_' + str(t)
            vd[path_rnn_input_key] = torch.zeros(path_rnn_input_shape)
            
            if (t > 0):  # for non leaf variables, we need to propagate the gradient back
                vd[path_rnn_input_key].requires_grad = True
            
            
            # path hidden state output from P_U, initialized with just bandwidth at T_0, else copy
            path_h_state_key = 'path_h_state_' + str(t)
            if (t==0):
                vd[path_h_state_key] = torch.cat((bandwidth,torch.zeros(path_h_state_shape)), axis=1)
            else:
                path_h_state_key = 'path_h_state_' + str(t)
                path_h_prev = 'path_h_state_' + str(t-1)
                vd[path_h_state_key] = vd[path_h_prev]
            
            
            # path_hidden state sequence from P_U, used to update links
            path_h_state_seq_key = 'path_h_states_seq_' + str(t)
            
            
            ########## LINK VARIABLES ###########
            
            # vector to store the link_hidden states
            if (t == 0):
                # create hidden state matrix for links and initialize with first column as link capacity
                link_h_state_key = 'link_h_state_' + str(t)
                vd[link_h_state_key] = torch.cat((link_cap,torch.zeros(link_h_state_shape)),1)
            else:
                # copy hidden state value for next pass through
                link_h_state_key = 'link_h_state_' + str(t)
                linK_h_prev = 'link_h_state_' + str(t-1)
                vd[link_h_state_key] = vd[linK_h_prev]
                
                
            # matrix storing the hidden states of links on paths
            # i.e. the hidden state of all links in the x['links'] list
            h_link_path_key = 'h_links_on_paths_' + str(t)
            vd[h_link_path_key] = torch.gather(vd[link_h_state_key],0,indices.long())
            
            #link messages extracted from the path hidden state sequence output from P_U
            link_message_key = 'link_messages_' + str(t)

            
             # container for the link messages that are extracted from path rnn hidden states
            agg_link_message_key = 'aggregated_link_message_' + str(t)
            vd[agg_link_message_key] = torch.zeros((x['n_links'],self.link_state_dim),requires_grad=True)
            
            ########################################################################################
            
            
            ################################## DO THE MESSAGE PASSING ##############################
            
            # prepare input for path RNN
            vd[path_rnn_input_key] = vd[path_rnn_input_key].index_put(indices = [p_ind,s_ind],
                                                                      values = vd[h_link_path_key])
            # pass through the path RNN
            vd[path_h_state_seq_key], vd[path_h_state_key] = self.p_U(vd[path_rnn_input_key],
                                                                      torch.unsqueeze(vd[path_h_state_key],
                                                                      0))
            # reformat
            vd[path_h_state_key] = vd[path_h_state_key].squeeze(0)
            
            # extract link messages from the path RNN sequence output
            # equivalent to tf.gather_nd
            vd[link_message_key] = vd[path_h_state_seq_key][p_ind,s_ind,:]
           
            # aggregate the link messages
            vd[agg_link_message_key] = vd[agg_link_message_key].index_put([links.squeeze(1)],
                                                                           vd[link_message_key],
                                                                           accumulate=True)
            # update the state of the links by passing through link 
            _, vd[link_h_state_key] = self.l_U(torch.unsqueeze(vd[agg_link_message_key],1),
                                               torch.unsqueeze(vd[link_h_state_key].squeeze(0),0))
            # reformat
            vd[link_h_state_key] = vd[link_h_state_key].squeeze(0)
            
            ##########################################################################################

        # readout from the paths
        y = self.readout(vd[path_h_state_key])
        return y


    def readout(self,path_state):
        x = F.relu(self.readOut['r1'](path_state))
        x = F.relu(self.readOut['r2'](x))
        x = self.readOut['r3'](x)
        return x

In [8]:
model = RouteNet()

In [9]:
yy = model(x)

In [10]:
train(model,x,y,x['packets'])

[12.940240287543118]
[10.864437724147267]
[10.393563873358204]
[9.686747783363767]
[8.70490858703519]
[8.040384995506853]
[7.4026818804736925]
[6.799327639681752]
[6.232082177121792]
[5.719573609463538]
[5.27473293334764]
[4.902551195836412]
[4.555319747538169]
[4.253224082751974]
[3.9972590206604033]
[3.7555454340576038]
[3.5240834901789313]
[3.302503538977329]
[3.092297977460238]
[2.8942203016111656]
[2.7141375909380367]
[2.551136908399257]
[2.4003331377364465]
[2.2617579222544006]
[2.1339041467453423]
[2.0139331985443216]
[1.9010923814462812]
[1.7947756233239576]
[1.6946331114502715]
[1.600378221317224]
[1.511728041504864]
[1.4291904480500075]
[1.3518959030301283]
[1.2793083700509928]
[1.2111210934913699]
[1.148032716753248]
[1.0891574885704052]
[1.0338307865024126]
[0.9818163471028265]
[0.932897848509977]
[0.8868770322836436]
[0.8435712782832505]
[0.802811444334732]
[0.7644404774014717]
[0.7283117321064227]
[0.6942878855341322]
[0.6622405945750403]
[0.632049392691]
[0.6036006677053